In [4]:
%pip install -qU langchain langchain-ollama langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.embeddings import FakeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
# Step 1: Initialize Ollama Chat Model (NEW IMPORT)
llm = ChatOllama(
    model="qwen2.5:7b",
    temperature=0.7,
)

In [7]:
# Step 2: Prepare Documents
documents = [
    "LangChain is a framework for building applications with LLMs.",
    "It provides tools for agents, memory, and retrieval-augmented generation.",
    "Ollama lets you run open-source models locally on your machine.",
    "Python is a versatile programming language for AI and data science.",
]

In [8]:
# Step 3: Split Documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
)
chunks = text_splitter.split_text("\n".join(documents))

In [12]:
# Step 4: Create Vector Store
embeddings = FakeEmbeddings(size=256, model_name="fake-embedding")
vectorstore = InMemoryVectorStore.from_texts(chunks, embeddings)
retriever = vectorstore.as_retriever(k=2)

In [13]:
# Step 5: Define RAG Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer based on the context.\n\nContext: {context}"),
    ("human", "{query}"),
])

In [14]:
# Step 6: Simple RAG Function
def retrieve_and_respond(query: str) -> str:
    """Retrieve docs → format prompt → invoke LLM"""
    context_docs = retriever.invoke(query)
    context = "\n".join([doc.page_content for doc in context_docs])
    
    messages = prompt.format_messages(context=context, query=query)
    response = llm.invoke(messages)
    return response.content

In [15]:
# Step 7: Multi-turn Conversation
class ConversationPipeline:
    def __init__(self, llm, retriever, prompt):
        self.llm = llm
        self.retriever = retriever
        self.prompt = prompt
        self.chat_history = []
    
    def chat(self, user_query: str) -> str:
        """Handle multi-turn conversation"""
        context_docs = self.retriever.invoke(user_query)
        context = "\n".join([doc.page_content for doc in context_docs])
        
        # Format messages with context and conversation history
        messages = prompt.format_messages(context=context, query=user_query)
        
        response = self.llm.invoke(messages)
        response_text = response.content
        
        # Store in history
        self.chat_history.append({"user": user_query, "assistant": response_text})
        
        return response_text

In [16]:
# Step 8: Run Pipeline
if __name__ == "__main__":
    print("=== Single-turn RAG ===")
    result = retrieve_and_respond("What is LangChain?")
    print(f"Response: {result}\n")
    
    print("=== Multi-turn Conversation ===")
    conversation = ConversationPipeline(llm, retriever, prompt)
    
    queries = [
        "What is LangChain?",
        "Can I use it with Python?",
        "How does it relate to Ollama?"
    ]
    
    for query in queries:
        response = conversation.chat(query)
        print(f"User: {query}")
        print(f"Assistant: {response}\n")

=== Single-turn RAG ===
Response: LangChain is a framework designed to help build applications that utilize large language models (LLMs). It provides tools, functionalities, or methods for integrating agents, memory systems, and retrieval-augmented generation techniques with LLMs. This enables developers to create more sophisticated and context-aware AI applications using these advanced language models.

=== Multi-turn Conversation ===
User: What is LangChain?
Assistant: LangChain is a framework designed to facilitate the development of applications that utilize large language models (LLMs). It provides tools, including support for agents, memory, and retrieval-augmented generation, which help in creating more sophisticated and context-aware AI systems. By leveraging LangChain, developers can build applications that not only generate text based on LLMs but also incorporate additional functionalities such as contextual understanding and the ability to retain and utilize information acro